In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.insert(0,'../')

import requests
from tqdm.notebook import tqdm

In [3]:
import pymongo
import pprint
import json
from src.mongo_utils import collection
from src.mlflow_setup import mlflow
from src.ha_utils import set_sensor_state, Input


api_endpoint = f"http://127.0.0.1:8000/mixtral/"

In [6]:
from src.mongo_utils import get_database_entry, create_set


with open('../test_set1.json') as file:
    test_set_data = json.load(file)
test_set = [a for subl in test_set_data.values() for a in subl]

In [7]:
dataset = []
for data in test_set:
    dataset += create_set(get_database_entry(data))

In [4]:
with open('../test_set1_postprocessed.json', 'r') as file:
    dataset = json.load(file)

In [7]:
import random
random.seed(42)
random.shuffle(dataset)

In [5]:
import pandas as pd

eval_data = {
    "inputs": [],
    "ground_truth": [],
    "prompt_data": []
}
for title, context_word, context_sentence, gt in dataset:
    test = {}
    test['title'] = title
    test['context_word'] = context_word
    test['context_sentence'] = context_sentence.replace("'", "")
    eval_data['inputs'] += [f"{context_word} --- {context_sentence}"]
    eval_data['prompt_data'] += [test]
    eval_data['ground_truth'] += [gt]
eval_data = pd.DataFrame(eval_data)
eval_data

In [6]:
import ipywidgets as widgets
out = widgets.Output(layout={'border': '1px solid black'})

In [38]:
def eval_network(data, experiment, out=None):
    answers = []
    pbar = tqdm(data.iterrows(), total=len(data))
    set_sensor_state(pbar.n, pbar.total)
    for index, row in pbar:
        set_sensor_state(pbar.n, pbar.total)
        context_sentence = row['prompt_data']['context_sentence']
        context_word = row['prompt_data']['context_word']
        body = {
          "system_prompts": experiment.system_prompt,
          "dialog": [
              *experiment.create_examples(),
            {
              "role": "user",
              "content": experiment.question_prompt % (context_sentence, context_word)
            }
          ]
        }
        response = ""
        with requests.post(api_endpoint, json=body, stream=True) as r:
            for chunk in r.iter_content(1024):
                response += chunk.decode()
        answers.append(response)
        if out:
            with out:
                out.clear_output()
                print(f"{context_word}\n{context_sentence}\n\n{response}")
    set_sensor_state(pbar.n, pbar.total)
    return answers

In [22]:
out

In [23]:
class Experiment:
    def __init__(self, system_prompt=None, question_prompt="\"%s\": Was ist die Definition von %s?", example_prompts=[("Die Liebe überwindet alle Grenzen", "Liebe", "inniges Gefühl der Zuneigung für jemanden oder für etwas")]):
        if system_prompt:
            self.system_prompt = system_prompt
        else:
            self.system_prompt = ["Du bist ein Definitionsgenerator. "
                "Du antwortest in Deutsch. "
                "Du bekommst einen Beispielsatz und ein Wort aus dem Satz. "
                "Du antwortest mit der Definition des Wortes im Kontext des Beispiels. "
                "Du antwortest nur mit der Definition. "
                "Du zeigst den Bezug auf den Kontext. "
                "Du antwortest in maximal 5 Worten. "]
        self.question_prompt = question_prompt
        self.example_prompts = example_prompts

    def create_examples(self):
        ret_val = []
        for example, word, definition in self.example_prompts:
            ret_val += [{
              "role": "user",
              "content": self.question_prompt % (example, word)
            },
            {
              "role": "assistant",
              "content": definition}
           ]
        return ret_val

In [36]:
    Experiment(system_prompt=[
        "Du bist ein Definitionsgenerator. "
        "Du antwortest in Deutsch. "
        "Du bekommst einen Beispielsatz und ein Wort aus dem Satz. "
        "Du antwortest mit der Definition des Wortes im Kontext des Beispiels. "
        "Du antwortest nur mit der Definition. "
        "Du zeigst den Bezug auf den Kontext. "
        "Du antwortest in maximal 5 Worten. "
      ], 
               question_prompt="\"%s\": Was ist die Definition von %s?", 
               example_prompts=[("Die Liebe überwindet alle Grenzen", "Liebe", "inniges Gefühl der Zuneigung für jemanden oder für etwas")]),


experiments = [

    Experiment(system_prompt=[
        "Du bist ein Definitionsgenerator. "
        "Du antwortest in Deutsch. "
        "Du bekommst einen Beispielsatz und ein Wort aus dem Satz. "
        "Du antwortest mit der Definition des Wortes im Kontext des Beispiels. "
        "Du antwortest nur mit der Definition. "
        "Du zeigst den Bezug auf den Kontext. "
        "Du antwortest in maximal 5 Worten. "
      ], 
               question_prompt="\"%s\": Was ist in diesem Kontext die Definition von %s?", 
               example_prompts=[
                   ("Die Liebe überwindet alle Grenzen", "Liebe", "inniges Gefühl der Zuneigung für jemanden oder für etwas"),
                   ("Natürlich sind diese Stifte stabil und robust genug, um den täglichen Rettungseinsatz absolvieren zu können.", "Stifte", "Rekrut"),
               ])
]

In [39]:
exp_tqdm = tqdm(experiments)

for exp in exp_tqdm:
    set_sensor_state(exp_tqdm.n, exp_tqdm.total, Input.TOTAL_INPUT)
    with mlflow.start_run():
    
        mlflow.log_param("system_prompt", exp.system_prompt)
        mlflow.log_param("question_prompt", exp.question_prompt)
        mlflow.log_param("example_prompt", exp.example_prompts)
    
        results = mlflow.evaluate(
            lambda x: eval_network(x, exp, out),
            eval_data,
            model_type="text",
            targets="ground_truth",
            extra_metrics=[mlflow.metrics.rougeL()]#, bert_f1_score_metric]
        )
set_sensor_state(exp_tqdm.n, exp_tqdm.total, Input.TOTAL_INPUT)
